In [ ]:
import torch
from torch import nn, optim
from torchvision.transforms import Resize, ToTensor, Compose
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
from pathlib import Path
import random
import matplotlib.pyplot as plt

# Configuration de l'appareil
device = "cuda" if torch.cuda.is_available() else "cpu"

# Définition des transformations
train_transform = Compose([
    Resize((224, 224)), 
    ToTensor()
])

# Création du Dataset et du DataLoader
data_dir = Path('/kaggle/input/imagenetmini-1000/imagenet-mini')

# Dataset d'entraînement et de validation
training_dataset = ImageFolder(root=data_dir / 'train', transform=train_transform)

# Diviser les données en 80% entraînement et 20% validation
train_size = int(0.8 * len(training_dataset))
val_size = len(training_dataset) - train_size
train_dataset, val_dataset = random_split(training_dataset, [train_size, val_size])

train_dataloader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)

In [ ]:
# Définition du modèle AlexNet
class AlexNet(nn.Module):
    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4),  # Conv1
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.LocalResponseNorm(5),
            
            nn.Conv2d(96, 256, kernel_size=5, padding=2),  # Conv2
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.LocalResponseNorm(5),
            
            nn.Conv2d(256, 384, kernel_size=3, padding=1),  # Conv3
            nn.ReLU(inplace=True),
            
            nn.Conv2d(384, 384, kernel_size=3, padding=1),  # Conv4
            nn.ReLU(inplace=True),
            
            nn.Conv2d(384, 256, kernel_size=3, padding=1),  # Conv5
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(self._get_flattened_size(), 4096),  # FC6
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),  # FC7
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes)  # FC8
        )
    def _get_flattened_size(self):
        with torch.no_grad():
            dummy_input = torch.randn(1, 3, 224, 224)  # Create a dummy input tensor
            features_output = self.features(dummy_input)  # Pass through the features part
            return features_output.view(1, -1).size(1)  # Get the flattened size
            
    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)  # Aplatir toutes les dimensions sauf la première
        x = self.classifier(x)
        return x

In [ ]:
# Initialisation du modèle, de la fonction de perte et de l'optimiseur
alexnet_model = AlexNet(num_classes=len(training_dataset.classes)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(alexnet_model.parameters(), lr=0.001)

# Vérification : le modèle est-il sur le GPU ?
print("Modèle sur l'appareil:", next(alexnet_model.parameters()).device)

# Fonction pour évaluer le modèle
def evaluate_model(dataloader):
    alexnet_model.eval()  # Mettre le modèle en mode évaluation
    total_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = alexnet_model(images)
            loss = criterion(outputs, labels)

            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    average_loss = total_loss / len(dataloader)
    accuracy = correct / total
    return average_loss, accuracy

# Boucle d'entraînement avec validation
num_epochs = 10
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

for epoch in range(num_epochs):
    alexnet_model.train()  # Mettre le modèle en mode entraînement
    running_loss = 0.0

    for images, labels in train_dataloader:
        images, labels = images.to(device), labels.to(device)
        
        # Réinitialiser les gradients
        optimizer.zero_grad()
        
        # Passage avant
        outputs = alexnet_model(images)
        
        # Calcul de la perte
        loss = criterion(outputs, labels)
        
        # Passage arrière et optimisation
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    # Évaluation du modèle sur l'ensemble d'entraînement
    train_loss, train_accuracy = evaluate_model(train_dataloader)
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)

    # Évaluation du modèle sur l'ensemble de validation
    val_loss, val_accuracy = evaluate_model(val_dataloader)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_dataloader):.4f}, '
          f'Training Accuracy: {train_accuracy:.4f}, Validation Accuracy: {val_accuracy:.4f}')

# Visualisation des pertes et des précisions
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss', linestyle='--')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Training Accuracy', color='orange')
plt.plot(val_accuracies, label='Validation Accuracy', color='green', linestyle='--')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.show()
